Attribution: Code borrowed from https://github.com/kuangliu/pytorch-cifar

In [107]:
# install required packages
!pip3 install torch
!pip3 install torchvision
!pip3 install torchsummary
!pip3 install matplotlib

In [108]:
# import required packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from models import *
from torchsummary import summary
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [109]:
# hyperparamters to adjust
model = "MobileNetV2"
lr = 0.1
epochs = 10
batch_size = 128
optimizer = "SGD"
has_data_aug = 1

filename = model + "_" + str(lr) + "_" + str(epochs) + "_" + str(batch_size) + "_" + optimizer + "_" + str(data_aug)
# Example file format
print(filename)

MobileNetV2_0.1_10_128_SGD_1


In [110]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0
start_epoch = 0

In [111]:
# Data
if has_data_aug:
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [112]:
# Model

# net = VGG('VGG19')      20,040,522
# net = ResNet18()        11,173,962
# net = PreActResNet18()  11,171,146
# net = GoogLeNet()        6,166,250
# net = DenseNet121()      6,956,298
# net = ResNeXt29_2x64d()  9,128,778
# net = MobileNet()        3,217,226
# net = MobileNetV2()      2,296,922
# net = DPN92()           34,236,634
# net = ShuffleNetG2()     n/a
# net = SENet18()         11,260,354
# net = ShuffleNetV2(1)    1,263,854
# net = EfficientNetB0()   3,598,598
# net = RegNetX_200MF()    2,321,946
# net = SimpleDLA()       15,142,970

net = MobileNetV2()
# print(net)
# print(summary(net, (3, 32, 32)))

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# commented args logic for notebook
# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint/ckpt.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [113]:
# Training
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return train_loss/len(trainloader), correct/total

In [114]:
# Testing
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/' + filename + '.pth')
        best_acc = acc

    return test_loss/len(testloader), correct/total

In [115]:
def plotLoss():
    plt.plot(range(epochs), history['train_loss'], '-', linewidth=3, label='Train Loss')
    plt.plot(range(epochs), history['test_loss'], '-', linewidth=3, label='Test Loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid(True)
    plt.legend()
    plt.show()

In [116]:
def plotAcc():
    plt.plot(range(epochs), history['train_acc'], '-', linewidth=3, label='Train Acc')
    plt.plot(range(epochs), history['test_acc'], '-', linewidth=3, label='Test Acc')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
train_loss_history = []
test_loss_history = []
train_acc_history = []
test_acc_history = []

history = {
    'train_loss': [],
    'test_loss': [],
    'train_acc': [],
    'test_acc': []
}

for epoch in range(start_epoch, start_epoch+epochs):

    print('\nEpoch: %d' % epoch)
    train_loss, train_acc = train(epoch)

    print("Train \tLoss: %.3f | Acc: %.3f" % (train_loss, train_acc))

    test_loss, test_acc = test(epoch)
    # print('Test')
    print("Test \tLoss: %.3f | Acc: %.3f" % (test_loss, test_acc))

    history['train_loss'].append(train_loss)
    history['test_loss'].append(test_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)

    scheduler.step()



Epoch: 0
Train 	Loss: 1.825 | Acc: 0.339
acc 41.23
best_acc 0
Saving..
Test 	Loss: 1.588 | Acc: 0.412

Epoch: 1
Train 	Loss: 1.288 | Acc: 0.530
acc 60.54
best_acc 41.23
Saving..
Test 	Loss: 1.080 | Acc: 0.605

Epoch: 2
Train 	Loss: 1.011 | Acc: 0.640
acc 65.05
best_acc 60.54
Saving..
Test 	Loss: 0.988 | Acc: 0.650

Epoch: 3
Train 	Loss: 0.850 | Acc: 0.703
acc 73.09
best_acc 65.05
Saving..
Test 	Loss: 0.769 | Acc: 0.731

Epoch: 4
Train 	Loss: 0.755 | Acc: 0.738
acc 72.06
best_acc 73.09
Test 	Loss: 0.825 | Acc: 0.721

Epoch: 5
Train 	Loss: 0.690 | Acc: 0.760
acc 75.82
best_acc 73.09
Saving..
Test 	Loss: 0.713 | Acc: 0.758

Epoch: 6
Train 	Loss: 0.650 | Acc: 0.776
acc 75.59
best_acc 75.82
Test 	Loss: 0.704 | Acc: 0.756

Epoch: 7
Train 	Loss: 0.631 | Acc: 0.785
acc 71.86
best_acc 75.82
Test 	Loss: 0.816 | Acc: 0.719

Epoch: 8
Train 	Loss: 0.630 | Acc: 0.784
acc 78.52
best_acc 75.82
Saving..
Test 	Loss: 0.634 | Acc: 0.785

Epoch: 9


In [ ]:
# save history dict as json

with open("./results/" + filename + ".json", "w") as outfile:
    json.dump(history, outfile)

In [ ]:
plotAcc()

In [ ]:
plotLoss()